In [59]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import os
import us
import geocoder as geo
plotly.offline.init_notebook_mode(connected=True)

inpatient_files = []
outpatient_files = []

for direc,b,files in os.walk("./data/"):
    inpatient_files.extend([direc+"/"+x for x in files if "Inpatient" in x and ".csv" in x])
    outpatient_files.extend([direc+"/"+x for x in files if "Outpatient" in x and ".csv" in x])
        
inpatient_files, outpatient_files

(['./data//Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv',
  './data//Medicare_Provider_Charge_Inpatient_DRG100_FY2012.csv',
  './data//Medicare_Provider_Charge_Inpatient_DRG100_FY2013.csv',
  './data//Medicare_Provider_Charge_Inpatient_DRGALL_FY2014.csv',
  './data//Medicare_Provider_Charge_Inpatient_DRGALL_FY2015.csv'],
 ['./data//Medicare_Charge_Outpatient_APC28_CY2015_Provider.csv',
  './data//Medicare_Provider_Charge_Outpatient_APC30_CY2011_v2.csv',
  './data//Medicare_Provider_Charge_Outpatient_APC30_CY2012.csv',
  './data//Medicare_Provider_Charge_Outpatient_APC30_CY2013_v2.csv',
  './data//Medicare_Provider_Charge_Outpatient_APC32_CY2014.csv'])

In [2]:
def getData(f_type, year):
    if f_type not in ['inpatient','outpatient']:
        raise Exception('Incorrect file type. Input "inpatient" or "outpatient"')
    if type(year) != int:
        raise Exception("Year but be integer.")
    elif year not in [2011, 2012, 2013, 2014, 2015]:
        raise Exception("Year must be between 2011 and 2015, inclusive.")
    if f_type == 'inpatient':
        inp_file = [x for x in inpatient_files if str(year) in x][0]
        return pd.read_csv(inp_file)
    else:
        out_file = [x for x in outpatient_files if str(year) in x][0]
        return pd.read_csv(out_file)
        
inp_2011 = getData("inpatient", 2011)
inp_2011.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07692,5777.241758,4763.736264
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85714,5787.571429,4976.714286
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37500,5434.958333,4453.791667
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28000,5417.560000,4129.160000
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27778,5658.333333,4851.444444


In [3]:
groupby_hosp_2011 = inp_2011[["DRG Definition","Provider Name","Average Covered Charges"]].groupby(["Provider Name","DRG Definition"]).mean()
groupby_hosp_2011.reset_index(inplace=True)
nationwide_drg_2011 = inp_2011[["DRG Definition","Average Total Payments"]].groupby(["DRG Definition"]).mean()

In [4]:
groupby_hosp_2011.iloc[0:10,:]

,Provider Name,DRG Definition,Average Covered Charges
0,ABBEVILLE GENERAL HOSPITAL,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,24959.58333
1,ABBEVILLE GENERAL HOSPITAL,192 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/...,17578.86667
2,ABBEVILLE GENERAL HOSPITAL,194 - SIMPLE PNEUMONIA & PLEURISY W CC,17579.66667
3,ABBEVILLE GENERAL HOSPITAL,195 - SIMPLE PNEUMONIA & PLEURISY W/O CC/MCC,13733.08696
4,ABBEVILLE GENERAL HOSPITAL,202 - BRONCHITIS & ASTHMA W CC/MCC,16665.56250
5,ABBEVILLE GENERAL HOSPITAL,291 - HEART FAILURE & SHOCK W MCC,27433.11111
6,ABBEVILLE GENERAL HOSPITAL,292 - HEART FAILURE & SHOCK W CC,16700.65385
7,ABBEVILLE GENERAL HOSPITAL,293 - HEART FAILURE & SHOCK W/O CC/MCC,11048.52632
8,ABBEVILLE GENERAL HOSPITAL,"392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DIS...",14184.41379
9,ABBEVILLE GENERAL HOSPITAL,470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,30701.07692


In [5]:
print(nationwide_drg_2011.shape)
nationwide_drg_2011.iloc[0:10,:]

(100, 1)


,Average Total Payments
DRG Definition,
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,6960.537900
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,6706.280366
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC,13263.827101
065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC,7922.675347
066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W/O CC/MCC,5713.989142
069 - TRANSIENT ISCHEMIA,5068.648086
074 - CRANIAL & PERIPHERAL NERVE DISORDERS W/O MCC,6386.796901
101 - SEIZURES W/O MCC,5493.365845
149 - DYSEQUILIBRIUM,4589.107826


# What are the highest national average covered charges?

In [6]:
nationwide_drg_2011.sort_values("Average Total Payments")[:-10:-1]

,Average Total Payments
DRG Definition,
870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,44259.489776
853 - INFECTIOUS & PARASITIC DISEASES W O.R. PROCEDURE W MCC,40315.965619
207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR SUPPORT 96+ HOURS,38588.924930
329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC,37765.598323
460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC,27778.675288
246 - PERC CARDIOVASC PROC W DRUG-ELUTING STENT W MCC OR 4+ VESSELS/STENTS,23326.342987
252 - OTHER VASCULAR PROCEDURES W MCC,22845.643322
469 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W MCC,22531.265543
238 - MAJOR CARDIOVASC PROCEDURES W/O MCC,21948.549102


# Lets look at difference between the the charges incurred by hospitals and the amount paid out by CMS

In [7]:
merged_df = groupby_hosp_2011.merge(nationwide_drg_2011, on='DRG Definition')
merged_df.loc[:,'diff'] = merged_df['Average Covered Charges'] - merged_df['Average Total Payments']
merged_df.sort_values('diff',ascending=False).iloc[:10,:]

,Provider Name,DRG Definition,Average Covered Charges,Average Total Payments,diff
70648,STANFORD HOSPITAL,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,929118.9091,38588.924930,890529.984170
70774,WASHINGTON HOSPITAL,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,918023.1818,38588.924930,879434.256870
146174,STANFORD HOSPITAL,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,637377.7143,44259.489776,593118.224524
145892,NORTHBAY MEDICAL CENTER,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,628730.4000,44259.489776,584470.910224
70666,TEMPLE UNIVERSITY HOSPITAL,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,613926.6071,38588.924930,575337.682170
70184,MEADOWLANDS HOSPITAL MEDICAL CENTER,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,602981.3784,38588.924930,564392.453470
69824,CEDARS-SINAI MEDICAL CENTER,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,596878.0263,38588.924930,558289.101370
145548,CROZER CHESTER MEDICAL CENTER,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,596410.2683,44259.489776,552150.778524
69886,CROZER CHESTER MEDICAL CENTER,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,581801.9211,38588.924930,543212.996170
69807,CAPITAL HEALTH MEDICAL CENTER - HOPEWELL,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,578212.7500,38588.924930,539623.825070


In [8]:
highest_divergent_drgs = merged_df[['DRG Definition','diff']].groupby('DRG Definition').mean().sort_values("diff",ascending=False)
div_drg_upper = highest_divergent_drgs.iloc[:10,:]
div_drg_lower = highest_divergent_drgs.iloc[-10:,:]
div_drg_upper, div_drg_lower

(                                                             diff
 DRG Definition                                                   
 870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS    118920.460917
 207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR...  104740.064463
 853 - INFECTIOUS & PARASITIC DISEASES W O.R. PR...   99229.828885
 329 - MAJOR SMALL & LARGE BOWEL PROCEDURES W MCC     97529.306069
 246 - PERC CARDIOVASC PROC W DRUG-ELUTING STENT...   73074.341453
 460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC          67943.535772
 238 - MAJOR CARDIOVASC PROCEDURES W/O MCC            63074.938868
 252 - OTHER VASCULAR PROCEDURES W MCC                60746.310593
 469 - MAJOR JOINT REPLACEMENT OR REATTACHMENT O...   59170.679651
 480 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT...   54458.636187,
                                                             diff
 DRG Definition                                                  
 918 - POISONING & TOXIC EFFECTS OF DRUGS W/O MCC    12700.2238

In [9]:
upper_trace = go.Bar(
    x = highest_divergent_drgs['diff'][:10][::-1],
    y = highest_divergent_drgs.index[:10][::-1],
    orientation='h',
    marker=dict(
        color='red'
    )
)

data = [upper_trace]

layout = go.Layout(
    xaxis = dict(
        domain=[0.7,1],
    ),
    yaxis = dict(
        tickfont=dict(
            size=10
        ),
    ), 
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

These are the top 10 most expensive DRG codes covered by medicare. The amounts you see there are the dollar values that are leftover after medicare reimbursement. Who pays for these after medicare?

As for the codes themselves, no real surprises. Septicimia is an extremely costly to treat, mostly hospital contracted complication. The others seem to be surgical procedures with major complications.

# Lets take a look at some hospitals now. I want to identify the hospitals that price above and below the national average.

In [83]:
groupby_hosp_2011 = inp_2011[["DRG Definition","Provider Name","Average Covered Charges",'Provider Street Address','Provider City',"Hospital Referral Region (HRR) Description"]].groupby(["Provider Name","DRG Definition"]).mean().reset_index()
hosp_merge = groupby_hosp_2011.merge(nationwide_drg_2011, on='DRG Definition')
hosp_merge.loc[:,'diff'] = hosp_merge['Average Covered Charges'] - hosp_merge['Average Total Payments']

In [84]:
groupby_hosp_2011_wloc

,Provider Name,DRG Definition,Average Covered Charges
0,ABBEVILLE GENERAL HOSPITAL,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,24959.58333
1,ABBEVILLE GENERAL HOSPITAL,192 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/...,17578.86667
2,ABBEVILLE GENERAL HOSPITAL,194 - SIMPLE PNEUMONIA & PLEURISY W CC,17579.66667
3,ABBEVILLE GENERAL HOSPITAL,195 - SIMPLE PNEUMONIA & PLEURISY W/O CC/MCC,13733.08696
4,ABBEVILLE GENERAL HOSPITAL,202 - BRONCHITIS & ASTHMA W CC/MCC,16665.56250
5,ABBEVILLE GENERAL HOSPITAL,291 - HEART FAILURE & SHOCK W MCC,27433.11111
6,ABBEVILLE GENERAL HOSPITAL,292 - HEART FAILURE & SHOCK W CC,16700.65385
7,ABBEVILLE GENERAL HOSPITAL,293 - HEART FAILURE & SHOCK W/O CC/MCC,11048.52632
8,ABBEVILLE GENERAL HOSPITAL,"392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DIS...",14184.41379
9,ABBEVILLE GENERAL HOSPITAL,470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,30701.07692


In [85]:
hosp_merge[hosp_merge['diff'] < 0].sort_values('diff').iloc[:5]

,Provider Name,DRG Definition,Average Covered Charges,Average Total Payments,diff
144406,FREDERICK MEMORIAL HOSPITAL,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,28160.46429,40315.965619,-12155.501329
144759,MONTGOMERY GENERAL HOSPITAL INC,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,30574.47368,40315.965619,-9741.491939
144166,BON SECOURS HOSPITAL,853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,30812.14286,40315.965619,-9503.822759
144700,"MERCY HEALTH PARTNERS, HACKLEY CAMPUS",853 - INFECTIOUS & PARASITIC DISEASES W O.R. P...,31042.85714,40315.965619,-9273.108479
22150,CHICKASAW NATION MEDICAL CENTER,470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT ...,5303.60000,14566.933731,-9263.333731


In [86]:
hosp_merge[hosp_merge['diff'] > 0].sort_values('diff',ascending=False).iloc[:5]

,Provider Name,DRG Definition,Average Covered Charges,Average Total Payments,diff
70648,STANFORD HOSPITAL,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,929118.9091,38588.924930,890529.984170
70774,WASHINGTON HOSPITAL,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,918023.1818,38588.924930,879434.256870
146174,STANFORD HOSPITAL,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,637377.7143,44259.489776,593118.224524
145892,NORTHBAY MEDICAL CENTER,870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS,628730.4000,44259.489776,584470.910224
70666,TEMPLE UNIVERSITY HOSPITAL,207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,613926.6071,38588.924930,575337.682170


# Lets map these differences by county.

First we get the geo data and enable the capacity to map provider names to their county FIPs number.

In [119]:
#hosp_loc_dict = getData('inpatient',2011)[['Provider Name','Hospital Referral Region (HRR) Description']].set_index('Provider Name').to_dict(orient='index')
statefips = pd.read_excel('./data/state-geocodes-v2016.xls',header=4)
countyfips = pd.read_excel('./data/all-geocodes-v2016.xlsx',header=3)
countyfips.columns = ['sum_level','state_code','county_code','county_sub_code','place_code','consol_city_code','area_name']
statefips.head()

,Region,Division,State (FIPS),Name
0,1,0,0,Northeast Region
1,1,1,0,New England Division
2,1,1,9,Connecticut
3,1,1,23,Maine
4,1,1,25,Massachusetts


In [131]:
statefips.columns

Index(['Region', 'Division', 'State (FIPS)', 'Name'], dtype='object')

In [120]:
countyfips.head()

,sum_level,state_code,county_code,county_sub_code,place_code,consol_city_code,area_name
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County


### I lost my locations for the hospitals somewhere, lets get those back

In [108]:
hosp_merge_wloc = hosp_merge.merge(inp_2011[['Provider Name','Provider Street Address','Provider City','Provider State','Provider Zip Code','Hospital Referral Region (HRR) Description']],on='Provider Name',how='left').drop_duplicates(['Provider Name','DRG Definition'])
hosp_merge_wloc.head()

,Provider Name,DRG Definition,Average Covered Charges,Average Total Payments,diff,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description
0,ABBEVILLE GENERAL HOSPITAL,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,24959.58333,9800.381239,15159.202091,118 N HOSPITAL DR,ABBEVILLE,LA,70510,LA - Lafayette
18,ABBOTT NORTHWESTERN HOSPITAL,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,35716.41667,9800.381239,25916.035431,800 EAST 28TH STREET,MINNEAPOLIS,MN,55407,MN - Minneapolis
115,ABILENE REGIONAL MEDICAL CENTER,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,56629.83333,9800.381239,46829.452091,6250 HWY 83/84,ABILENE,TX,79606,TX - Abilene
173,ABINGTON MEMORIAL HOSPITAL,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,57558.35616,9800.381239,47757.974921,1200 OLD YORK ROAD,ABINGTON,PA,19001,PA - Philadelphia
272,ADENA REGIONAL MEDICAL CENTER,178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC,23704.52941,9800.381239,13904.148171,272 HOSPITAL ROAD,CHILLICOTHE,OH,45601,OH - Columbus


In [152]:
def fipsMapper(row):
    address = row.loc['Provider Street Address']
    city = row.loc['Provider City']
    state = str(us.states.lookup(row.loc['Provider State']))
    state_fip = statefips[statefips['Name']==state]['State (FIPS)'].values.tolist()[0]
    zipcode = str(row.loc['Provider Zip Code'])
    hrr = row.loc['Hospital Referral Region (HRR) Description']
    search_line = ', '.join([address,city,state,zipcode])
    search_result = geo.google(search_line)
    while search_result.county == None:
        search_result = geo.google(search_line)
    
    search_county = search_result.county
    county_fip = countyfips[(countyfips['area_name']==search_county) & (countyfips['state_code']==state_fip)].loc[:,'county_code'].values.tolist()
    if len(county_fip) > 0:
        return int(str(state_fip).zfill(2)+str(county_fip[0]).zfill(3))
    else:
        print(search_line)
    
hosp_merge_wloc[:10].apply(lambda row: fipsMapper(row),axis=1)

3815 HIGHLAND AVENUE, DOWNERS GROVE, Illinois, 60515


0      22113.0
18     27053.0
115    48441.0
173    42091.0
272    39141.0
362    17031.0
441     6031.0
554    17031.0
652    17097.0
748        NaN
dtype: float64